In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import HDF5Matrix 
import h5py
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
#f= h5py.File('Train/food_c101_n1000_r384x384x3.h5',

In [2]:
xtrain=HDF5Matrix('Train/food_c101_n10099_r32x32x1.h5','images')
ytrain=HDF5Matrix('Train/food_c101_n10099_r32x32x1.h5','category')

In [3]:
print ('xtrain shape is: ',xtrain.shape,', ytrain shape is: ',ytrain.shape)

xtrain shape is:  (10099, 32, 32, 1) , ytrain shape is:  (10099, 101)


In [4]:
xtrain=np.asarray(xtrain,dtype='float32')


In [5]:
ytrain=np.asarray(ytrain,dtype='float32')

In [6]:
xtest=HDF5Matrix('Test/food_test_c101_n1000_r32x32x1.h5','images')
ytest=HDF5Matrix('Test/food_test_c101_n1000_r32x32x1.h5','category')

In [7]:
print ('xtest shape is: ',xtest.shape,', ytest shape is: ',ytest.shape)

xtest shape is:  (1000, 32, 32, 1) , ytest shape is:  (1000, 101)


In [8]:
xtest=np.asarray(xtest,dtype='float32')
ytest=np.asarray(ytest,dtype='float32')

In [ ]:
labels =f.get('category')

In [ ]:
labels[:][:].shape

In [ ]:
labels=np.array(labels)

In [ ]:
header=np.array(f)

In [ ]:
header

In [ ]:
category_names =f.get('category_names')
category_names=np.array(category_names)

In [ ]:
category_names.shape

scale the data  

In [9]:
xtrain_scaled=xtrain/255
xtest_scaled=xtest/255

# encode the labels

In [ ]:
#from keras.utils import to_categorical

In [ ]:
#labels_encoded=to_categorical(labels)

In [ ]:
#labels_encoded=labels_encoded.astype('float32')

In [ ]:
#labels_encoded.T.shape

# Network params

In [10]:
n_channel=1
n_width=32
n_heght=32
n_output=101

In [ ]:
def create_variable(channel,width,height,output):
    X=tf.placeholder(tf.float32,shape=[None,height,width,channel],name='Input')
    Y=tf.placeholder(tf.float32,shape=[None,output],name='output')
    return X,Y

In [11]:
def initialize_variable():
    W1=tf.get_variable("W1",[4,4,1,8],tf.float32,tf.contrib.layers.xavier_initializer(seed=0))
    W2=tf.get_variable("W2",[4,4,8,16],tf.float32,tf.contrib.layers.xavier_initializer(seed=0))
    return W1,W2
    

In [ ]:
#Create mini-batch

def mini_batch(xdata,ydata,batch_size):
    x_size=xdata.shape[0]
    y_size=ydata.shape[0]
    for i in range(0,x_size,batch_size):
        yield xdata[i:i+batch_size],ydata[i:i+batch_size]
    

CNN Layers

In [ ]:
def create_layer(W1,W2,X,Y):
    #W1,W2=initialize_variable()
    #X,Y=create_variable(channel,width,height,output)
    conv1=tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')
    max_pool1=tf.nn.max_pool(tf.nn.relu(conv1),ksize=[1,2,2,1],strides=[1,1,1,1],padding='SAME')
    
    conv2=tf.nn.conv2d(max_pool1,W2,strides=[1,2,2,1],padding='SAME')
    max_pool2=tf.nn.max_pool(tf.nn.relu(conv2),ksize=[1,4,4,1],strides=[1,2,2,1],padding='SAME')
    
    max_pool2=tf.contrib.layers.flatten(max_pool2)
    out=tf.contrib.layers.fully_connected(max_pool2,n_output,activation_fn=None)
    
    return out

In [12]:
def compute_cost(out,Y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out,labels=Y))
    return cost

In [21]:
def model(features,labels,mode):
    learning_rate=0.05
    #tf.reset_default_graph()
    #tf.get_default_graph()
    
    
    
    #x,y=create_variable(n_channel,n_width,n_heght,n_output)
    x=tf.reshape(features['x'],[-1,n_width,n_heght,n_channel],name='input_layer')
    y=labels
    
    W1,W2=initialize_variable()
    
    conv1=tf.nn.conv2d(x,W1,strides=[1,1,1,1],padding='SAME')
    max_pool1=tf.nn.max_pool(tf.nn.relu(conv1),ksize=[1,2,2,1],strides=[1,1,1,1],padding='SAME')
    
    conv2=tf.nn.conv2d(max_pool1,W2,strides=[1,2,2,1],padding='SAME')
    max_pool2=tf.nn.max_pool(tf.nn.relu(conv2),ksize=[1,4,4,1],strides=[1,2,2,1],padding='SAME')
    
    max_pool2=tf.contrib.layers.flatten(max_pool2)
    out=tf.contrib.layers.fully_connected(max_pool2,n_output,activation_fn=None)
    
    
    predictions={
        'classes':tf.argmax(input=out,axis=1),
        'probability':tf.nn.softmax(out,name='softmax_tensor')
    }
    
    if mode==tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,predictions=predictions)
    
    #Loss 
    cost=compute_cost(out,y)
    
    #configure Train OS
    #global_step_tensor = tf.Variable(0, trainable=False, name='global_step',dtype=tf.int64)
    if mode==tf.estimator.ModeKeys.TRAIN:
        
        opt=tf.train.AdamOptimizer(learning_rate=learning_rate)
        
        train_op=opt.minimize(loss=cost,global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode=mode,loss=cost,train_op=train_op)
    
    # configure for eval
    
    #if mode=tf.estimator.ModeKeys.EVAL:    
    
    
    eval_metrics_ops={
        'accuracy':tf.metrics.accuracy(labels=tf.argmax(input=y,axis=1),predictions=predictions['classes'])
    }
    
    return tf.estimator.EstimatorSpec(mode=mode,loss=cost,eval_metric_ops=eval_metrics_ops)

In [22]:
#create 
food_nn=tf.estimator.Estimator(model_fn=model)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': 1, '_save_summary_steps': 100, '_model_dir': 'C:\\Users\\padarsh\\AppData\\Local\\Temp\\tmpp1nc_03d', '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None}


In [23]:
tensors_to_log = {"probability": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

In [24]:
#Train features
train_input_fn=tf.estimator.inputs.numpy_input_fn(x={'x':xtrain_scaled},y=ytrain,batch_size=100,num_epochs=10,shuffle=True)
food_nn.train(train_input_fn,steps=20000,hooks=[logging_hook])


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\padarsh\AppData\Local\Temp\tmpp1nc_03d\model.ckpt.
INFO:tensorflow:step = 1, loss = 4.61518
INFO:tensorflow:probability = [[ 0.00990523  0.00989151  0.0099004  ...,  0.00989687  0.00989526
   0.00990678]
 [ 0.00990859  0.00988348  0.00989991 ...,  0.0099074   0.00989035
   0.00991366]
 [ 0.00989916  0.0098895   0.00990765 ...,  0.00988975  0.00988819
   0.00991281]
 ..., 
 [ 0.00990736  0.00988769  0.00990389 ...,  0.00989988  0.00989701
   0.00991729]
 [ 0.00990093  0.00988836  0.00990191 ...,  0.00989342  0.00989517
   0.00990462]
 [ 0.00990542  0.00989101  0.0098975  ...,  0.00989738  0.00989415
   0.00990313]]
INFO:tensorflow:probability = [[ 0.01046643  0.01082369  0.00794437 ...,  0.01152376  0.00873653
   0.01107534]
 [ 0.01046643  0.01082369  0.00794437 ...,  0.01152376  0.00873653
   0.01107534]
 [ 0.01046643  0.01082369  0.00794437 ...,  0.01152376  0.00873653
   0.01107534]
 ..

In [20]:
val_input_fn=tf.estimator.inputs.numpy_input_fn(
    x={'x':xtest_scaled},
    y=ytest,
    num_epochs=1,
    shuffle=False
)
eval_results=food_nn.evaluate(input_fn=val_input_fn)


ValueError: Can not squeeze dim[1], expected a dimension of 1, got 101 for 'remove_squeezable_dimensions/Squeeze' (op: 'Squeeze') with input shapes: [?,101].

In [17]:
test_input_fn=tf.estimator.inputs.numpy_input_fn(
    x={'x':xtest_scaled},
    #y=ytest,
    #num_epochs=1,
    shuffle=False
)
eval_results=food_nn.predict(input_fn=test_input_fn)


In [18]:
for i,p in enumerate(eval_results):
    print ('Prediction %s: %s'%(i+1,p['classes'] ))

INFO:tensorflow:Restoring parameters from C:\Users\padarsh\AppData\Local\Temp\tmpb61uw256\model.ckpt-1010
Prediction 1: 67
Prediction 2: 67
Prediction 3: 67
Prediction 4: 67
Prediction 5: 67
Prediction 6: 67
Prediction 7: 67
Prediction 8: 67
Prediction 9: 67
Prediction 10: 67
Prediction 11: 67
Prediction 12: 67
Prediction 13: 67
Prediction 14: 67
Prediction 15: 67
Prediction 16: 67
Prediction 17: 67
Prediction 18: 67
Prediction 19: 67
Prediction 20: 67
Prediction 21: 67
Prediction 22: 67
Prediction 23: 67
Prediction 24: 67
Prediction 25: 67
Prediction 26: 67
Prediction 27: 67
Prediction 28: 67
Prediction 29: 67
Prediction 30: 67
Prediction 31: 67
Prediction 32: 67
Prediction 33: 67
Prediction 34: 67
Prediction 35: 67
Prediction 36: 67
Prediction 37: 67
Prediction 38: 67
Prediction 39: 67
Prediction 40: 67
Prediction 41: 67
Prediction 42: 67
Prediction 43: 67
Prediction 44: 67
Prediction 45: 67
Prediction 46: 67
Prediction 47: 67
Prediction 48: 67
Prediction 49: 67
Prediction 50: 67
Pre

In [25]:
ytrain[1]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], dtype=float32)